# Melodi Python SDK - Quick Start

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

In [2]:
from melodi.melodi_client import MelodiClient
from melodi.feedback.data_models import Feedback, FeedbackUpdateRequest
from melodi.threads.data_models import Message, Thread, ThreadsQueryParams
from melodi.users.data_models import User

## Setup Melodi client

In [3]:
import os

os.environ["MELODI_API_KEY"] = "<YOUR_KEY>"

client = MelodiClient(verbose=True)

## Create a Project

In [ ]:
new_project = client.projects.create("Melodi SDK Test")

print(new_project.id)

## Create a thread in Melodi

In [ ]:

melodiClient = MelodiClient(verbose=True)

threadExternalId = "external-thread-id-1"

externalUser = User(
    externalId="test-user-id-1",
    name="Test User",
    email="test-user@example.com",
    segments={
      "team": ["engineering"],
    }
)

message1 = Message(
    externalId="message1",
    role='user',
    content='Tell me a joke',
)

message2 = Message(
    externalId="message2",
    role='json_tool',
    type='json',
    jsonContent={
        "joke_setup": "Why did the chicken cross the road?",
        "joke_punchline": "To get to the other side!"
    },
    metadata={
        "metadatatest": "value2"
    },
)

message3 = Message(
    externalId="message3",
    role='assistant',
    content='Why did the chicken cross the road? To get to the other side!',
    metadata={
        "metadatatest": "value3"
    },
)

thread = Thread(
    projectName="Melodi SDK Test",
    externalId=threadExternalId,
    messages=[message1, message2, message3],
    externalUser=externalUser
)
melodi_thread = melodiClient.threads.create_or_update(thread)

print(melodi_thread)

## Add feedback to the thread

In [ ]:
feedback = Feedback(
    externalThreadId=threadExternalId,
    externalMessageId="message3",
    feedbackText="I don't understand",
    feedbackType="NEGATIVE",

    externalUser=User(
        externalId="user-giving-feedback-1",
        name="Mr. User",
        email="mruser@example.com"
    )
)

melodi_feedback = melodiClient.feedback.create(feedback)
print(melodi_feedback)

## Add feedback with attributes

In [ ]:
feedback_with_attributes = Feedback(
    externalThreadId=threadExternalId,
    externalMessageId="message3",
    feedbackText="This is so funny",

    # These attributes and options need to already exist in your organization
    # Or this call will fail
    attributes={
      "Humor Level": "High",
    }
)

melodi_feedback_with_attributes = melodiClient.feedback.create(feedback_with_attributes)
print(melodi_feedback_with_attributes)

## Update existing feedback

In [ ]:
# There are two ways to specify the feedback object you would like to update

# Using external ids, you must specify the external id for the thread, message and user.
# If there are multiple feedback objects for this message and this user, the request with fail with a 400.
feedbackUpdate = FeedbackUpdateRequest(
    externalThreadId=threadExternalId,
    externalMessageId="message3",
    externalUser=User(
        externalId="user-giving-feedback-1"
    ),
    feedbackText="This was updated via python",
    attributes={
        "Quality": "Good"
    }
)

updated_feedback = melodiClient.feedback.update(feedbackUpdate)
print(updated_feedback)

# Or, use the id from the feedback response object
feedbackUpdate2 = FeedbackUpdateRequest(
    id=melodi_feedback_with_attributes.id,
    feedbackText="This feedback has been updated"
)

updated_feedback2 = melodiClient.feedback.update(feedbackUpdate2)
print(updated_feedback2)

## Create or update a user

In [ ]:
user = User(
    externalId="test-user-id-2",
    name="Different User",
    email="differentuser@example.com",
    segments={
      "team": ["sales"],
    }
)

melodi_user = client.users.create_or_update(user)
print(melodi_user)

## Fetch threads with feedback

In [ ]:
queryParams = ThreadsQueryParams(
  projectId=new_project.id,
  hasFeedback=True, # Only return threads that have feedback associated
  includeFeedback=True, # Include the associated feedback objects with the returned threads
)

threadsWithFeedbackResponse = client.threads.get(queryParams)

print(threadsWithFeedbackResponse.count)

for thread in threadsWithFeedbackResponse.rows:
    for message in thread.messages:
        for feedbackAssocation in message.externalFeedbackAssociations:
            print(feedbackAssocation.externalFeedback.feedbackText)